In [116]:
import numpy as np
import cv2
from scipy import ndimage
from skimage.exposure import rescale_intensity
from numpy import pi

### Methods defined: Convolution, GaussianSmoothing, ImageGradient,NonMaximaSuppress, Thresholding, EdgeLinking

In [117]:
#Convolution from smoothing from prev assignment
def convolution(image, kernel):
    '''
    Convolution for RGB and Gray Images
    Limited to Square kernel of odd length
    '''
    kernel = np.flipud(np.fliplr(kernel))
    outputImage = np.zeros_like(image, dtype = "float32")
    l = int((kernel.shape[0]-1)/2)
    if len(image.shape) == 3:
        for k in range(3):
            timage = image[:,:,k]
            imagePadded = np.zeros((timage.shape[0]+2*l,timage.shape[1]+2*l))
            imagePadded[l:-l,l:-l] = timage
            for j in range(timage.shape[1]):
                for i in range(timage.shape[0]):
                    (outputImage[:,:,k])[j,i]= (kernel*imagePadded[j-l:j+l,i-l:i+l]).sum()
    else:
        imagePadded = np.zeros((image.shape[0]+2*l,image.shape[1]+2*l))
        imagePadded[l:-l,l:-l] = image
        for j in range(image.shape[1]):
            for i in range(image.shape[0]):
                outputImage[j,i] = (kernel*imagePadded[j:j+2*l+1,i:i+2*l+1]).sum()
    outputImage = rescale_intensity(outputImage, in_range=(0, 255))
    outputImage = (outputImage*255).astype("uint8")
    return outputImage

In [118]:
#Gaussian smoothing from prev assignment
def GaussianSmoothing(image, kernel_size, sigma):
    s = kernel_size//2
    x, y = np.mgrid[-s:s+1,-s:s+1]
    k = 1 / (2.0* pi * sigma**2)
    guassianKernel = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))*k
    outputImage = convolution(image, guassianKernel)
    return outputImage

#We calculate XGradient and YGradient using 
def ImageGradient(smoothedImage):
    y = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    x = np.array([[1, 2, 1], [0, 0, 0], [-1, -2,-1]])

    yGradient = convolution(smoothedImage, y)
    xGradient = convolution(smoothedImage, x)


    mag = np.hypot(xGradient, yGradient) # calculated Hypotenuse of gradients
    mag = (mag/mag.max()) 
    mag = mag.astype("float32") 
    theta = np.arctan2(yGradient, xGradient) #calcuting angle/direction of vector
    return (mag, theta)

def NonmaximaSuppress(mag, theta):

    # r = mag.shape[0]
    # c = mag.shape[1]
    r, c = mag.shape
    outputMag = np.zeros_like(mag, dtype = "float32")
    newTheta = mag
    newTheta[newTheta<0] += pi
    for x in range(1,r-1):
        for y in range(1,c-1):
            front,rear = 255,255
            if (0<= newTheta[x][y] < pi/8.0) or (7*pi/8.0 <= newTheta[x][y] <= pi ):
                front,rear = mag[x,y+1],mag[x,y-1]
            elif (pi/8.0<= newTheta[x][y] < 3*pi/8.0):
                front,rear = mag[x+1,y-1],mag[x-1,y+1]
            elif (3*pi/8.0<= newTheta[x][y] < 5*pi/8.0):
                front,rear = mag[x+1,y],mag[x-1,y]
            elif (5*pi/8.0<= newTheta[x][y] < 7*pi/8.0):
                front,rear = mag[x-1,y-1],mag[x+1,y+1]
            if mag[x,y] >= front and mag[x,y] >= rear:
                outputMag[x,y] = mag[x,y]
            else:
                outputMag[x,y] = 0
    # outputMag = (outputMag/outputMag.max())
    # outputMag = outputMag.astype("float32")
    return outputMag

In [119]:
def Thresholding(mag, lowThreshold, highThreshold):
    r = mag.shape[0]
    c = mag.shape[1]
    lowThreshold = mag.max()*lowThreshold
    highThreshold = mag.max()*highThreshold
    outputMagWeak = np.zeros_like(mag, dtype = "float32")
    outputMagStrong = np.zeros_like(mag, dtype = "float32")
    for x in range(0,r):
        for y in range(0,c):
            if(mag[x,y] > highThreshold):
                outputMagStrong[x,y] = 1
            elif (lowThreshold<= mag[x,y] <=highThreshold):
                outputMagWeak[x,y] = 0.3922
    # outputMagWeak = outputMagWeak.astype("uint8")
    # outputMagStrong = outputMagStrong.astype("uint8")
    return (outputMagWeak,outputMagStrong)

def edgeLinking(Mag_weak, Mag_strong):
    r = Mag_weak.shape[0]
    c = Mag_weak.shape[1]
    outputMag = Mag_strong[:,:]
    
    def recursivetraverse(x,y):
        if(Mag_weak[x,y]!=0):
            for i in range(-1,2):
                for j in range(-1,2):
                    if (Mag_strong[x+i,y+j]==0 and not(i==0 and j==0)):
                        Mag_strong[x+i,y+j] = 1
                        recursivetraverse(x+i,y+j)

    for x in range(1,r-1):
        for y in range(1,c-1):
            if Mag_strong[x,y]:
                Mag_strong[x,y] = 1
                recursivetraverse(x,y)
    outputMag = Mag_strong[:,:]
    return outputMag

### Working on Lena Image, with intermediate results shown

In [217]:

# Reading the lena/Test image as Gray Scale
im1 = cv2.imread("lena_gray.png",cv2.IMREAD_GRAYSCALE)
cv2.imshow('Orig',im1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [218]:

blurImage = GaussianSmoothing(im1, 3,1.5)
cv2.imshow('Blur',blurImage)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [219]:
(gradientImage, theta) = ImageGradient(blurImage)

cv2.imshow('Gradient',np.float32(gradientImage))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [220]:
imnonmaximaSuppress = NonmaximaSuppress(gradientImage, theta)
(MagWeak,MagStrong) = Thresholding(imnonmaximaSuppress, 0.02,0.08)
cv2.imshow('NonmaximaSuppress',imnonmaximaSuppress)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [221]:
imedgeLinking = edgeLinking(MagWeak, MagStrong)
cv2.imshow('edgeLinking',imedgeLinking)
cv2.waitKey(0)
cv2.destroyAllWindows()